In [ ]:
import numpy as np

In [ ]:
class Neuron:
    def __init__(self, weights, bias, lr):
        self.weights = weights
        self.bias = bias
        self.error = 0
        self.delta_weights = []
        self.delta_bias = 0
        self.lr = lr
    
    def calculate_neuron(self, inputs):
        """
        Calculate the output of this neuron
    
        """
        self.input = inputs
        list_outputs = []
        if len(inputs) == len(self.weights):
            for x, weight in zip(inputs, self.weights):
                list_outputs.append(x*weight)
        else:
            raise ValueError("The list of inputs and weights must have the same length")
        
        self.sum_output = sum(list_outputs)
        self.outcome = self.sum_output + self.bias
        output_sigmoid = self.sigmoid_function(self.outcome)
        return output_sigmoid
    
    def sigmoid_function(self, x):
        return 1/(1+np.exp(-x))
    
    def set_delta(self):
        for weight in self.weights:
            self.delta_weights.append(weight*self.error) # delta weight is lr * output --> 
        self.delta_bias = self.lr * self.error
        
    def get_deltas(self):
        # Function that returns a deltas on index for calculating next (previous) layer error
        deltas = []
        for weight in self.weights:
            deltas.append(weight*self.error)
        return deltas
    
    def update_neuron(self):
        for index in range(len(self.weights)):
            self.weights[index] -= self.delta_weights[index]*self.input[index]
        self.bias-= self.delta_bias
        
        self.delta_weights = []
    
class Output_neuron(Neuron):
    def __init__(self, weights, bias, lr):
        super().__init__(weights, bias, lr)
        self.what_am_i = 'im an output neuron.'

    def calculate_error(self, output_neuron, target):
        self.error = output_neuron * (1 - output_neuron) * - (target - output_neuron)

    def gradient(self, output):
        return output * self.error    
        
    def calculate_delta(self, output_neural_network):
        for weight in self.weights:
            self.delta_weights.append(self.lr * self.gradient(output_neural_network))
        self.delta_bias = self.lr * self.error
        
class Hidden_neuron(Neuron):
    def __init__(self, weights, bias, lr):
        super().__init__(weights, bias, lr)
        self.what_am_i = 'im an hidden neuron.'
         
    def calculate_error(self, deltas_weight):
        self.error += sum([delta*weight for delta, weight in zip(deltas_weight, self.weights)])
        
    def calculate_delta(self):
        for weight in self.weights:
            self.delta_weights.append(self.error * weight)
        self.delta_bias = self.bias * self.error

In [ ]:
class Neuron_layer:
    def __init__(self, neurons):
        self.neurons = neurons
    
    def calculate_layer(self, inputs):
        output = []
        for neuron in self.neurons:
            output.append(neuron.calculate_neuron(inputs))
        return output
    
    def set_output_layer(self, outputNN, target):
        for neuron in self.neurons:
            neuron.calculate_error(outputNN, target) # --> let op outputneuron functie wordt hier aangeroepen. -> check dit
            neuron.calculate_delta(outputNN)
    
    def set_hidden_layer(self, deltas_right_layer):
        for index in range(len(self.neurons)):
            print(self.neurons[index].what_am_i)
            self.neurons[index].calculate_error([deltas[index] for deltas in deltas_right_layer])
    
    def get_deltas_layer(self):
        deltas_layer = []
        for neuron in self.neurons:
            deltas_layer.append(neuron.get_deltas())
        return deltas_layer
        
    def update_layer(self):
        for neuron in self.neurons:
            neuron.update_neuron()
    
    def __str__(self):
        return f'The layer consist out of these neurons: {self.neurons}'

In [ ]:
class Neuron_network:
    def __init__(self, neuron_layers):
        self.neuron_layers = neuron_layers
    
    def feed_forward(self, inputs):
        for neuron_layer in self.neuron_layers:
            inputs = neuron_layer.calculate_layer(inputs)
        return inputs
    
    def __str__(self):
        return f'The network consist out of these perceptron_layers: {self.neuron_layers}'
    
    def MSE(self, target_list, output_list): # loss function 
        return np.sum([(a-b)**2 for a,b in zip(target_list, output_list)])
    
    def train_nn(self, posible_inputs):
        """
        Put in like this [[[0, 0], 0],
                          [[0, 1], 0],
                          [[1, 0], 0],
                          [[1, 1], 1]]
        """
        error = 1 # The error is unknown but will be assigned after the first check
        epoch = 0
        while error > 0.1: #epoch < 10: #
            li = []
            for posible_input in posible_inputs:  
                outputNN = self.feed_forward(posible_input[0])[0]
                li.append(outputNN)
            
                # backpropagation
                self.neuron_layers[-1].set_output_layer(outputNN, posible_input[1])
                
                for left_layer, right_layer in zip(self.neuron_layers[-2::-1], self.neuron_layers[-1::-1]): # Starting at -2 because last layer is output layer
                    deltas_next_layer = right_layer.get_deltas_layer()
                    print(deltas_next_layer)
                    left_layer.set_hidden_layer(deltas_next_layer)
                
                for layer in self.neuron_layers:
                    layer.update_layer()

            error = self.MSE([x[1] for x in posible_inputs], li)
            print('error = ', error)
            if error < 0.1:
                print('network trained!')
            
            epoch+=1

In [ ]:
and_neuron = Output_neuron([1,1], 1, 0.1)

output_layer = Neuron_layer([and_neuron])

neural_network_AND = Neuron_network([output_layer])

In [ ]:
#neural_network_AND.train_nn([[[0, 0], 0],
#                             [[0, 1], 0],
#                             [[1, 0], 0],
#                             [[1, 1], 1]])

In [ ]:
inputs = [[0,0],[0,1],[1,0],[1,1]]

print('x1, x2, carry, sum')
for input_x in inputs:
    print(input_x, '   ', neural_network_AND.feed_forward(input_x))

x1, x2, carry, sum
[0, 0]     [0.7310585786300049]
[0, 1]     [0.8807970779778823]
[1, 0]     [0.8807970779778823]
[1, 1]     [0.9525741268224334]


In [ ]:
nand_neuron_x = Hidden_neuron([1,1], 1, 0.1)
or_neuron_x = Hidden_neuron([1,1], 1, 0.1)
and_neuron_x = Output_neuron([1,1], 1, 0.1)

hidden_layer_x = Neuron_layer([nand_neuron_x, or_neuron_x])
output_layer_x = Neuron_layer([and_neuron_x])

neural_network_xor = Neuron_network([hidden_layer_x, output_layer_x])

In [ ]:
neural_network_xor.train_nn([[[0, 0], 0],
                             [[0, 1], 1],
                             [[1, 0], 1],
                             [[1, 1], 0]])

[[0.0666993502535941, 0.0666993502535941]]
im an hidden neuron.
im an hidden neuron.


IndexError: ignored